In [2]:
import pandas as pd
from scipy.optimize import minimize
import numpy as np
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage

In [3]:
df = pd.read_csv("../data/processed/processed_data.csv")
df.head()

,Date,U.S. 30 Day TBill TR,Russell 2000 TR,S&P 500 TR,LB LT Gvt/Credit TR,MSCI EAFE TR,Period
0,1980-01-31,0.80,8.420000,6.10,-6.14,4.84,1980s
1,1980-02-29,0.89,-1.800000,0.31,-7.92,-0.40,1980s
2,1980-03-31,1.21,-18.109999,-9.87,-0.43,-10.62,1980s
3,1980-04-30,1.26,6.270001,4.29,14.17,9.46,1980s
4,1980-05-31,0.81,8.340000,5.62,5.81,4.67,1980s


In [25]:
(df.drop(["Date","Period","U.S. 30 Day TBill TR "], axis=1).mean()*12).values

array([11.63497336, 12.02246045,  9.81895654, 11.90354087])

In [24]:
(df.drop(["Date","Period","U.S. 30 Day TBill TR "], axis=1).std()*12).values

array([68.08941865, 53.39633295, 42.06009394, 62.73631231])

In [14]:
df.drop(["Date","Period", "U.S. 30 Day TBill TR "], axis=1).cov().values

array([[32.19561759, 21.75402575,  2.56856258, 17.68042371],
       [21.75402575, 19.79978037,  3.20515945, 14.97543642],
       [ 2.56856258,  3.20515945, 12.28507988,  1.97896793],
       [17.68042371, 14.97543642,  1.97896793, 27.33225613]])

In [21]:
df["U.S. 30 Day TBill TR "].mean()*12

5.0529466779

In [16]:
df["U.S. 30 Day TBill TR "].std()

0.28742221564609893

In [17]:
0.8 * df["U.S. 30 Day TBill TR "].mean()

0.33686311186

In [20]:
0.8 * df["U.S. 30 Day TBill TR "].mean() + np.dot(np.array([0,0.1922,0.45,0.35]),df.drop(["Date","Period","U.S. 30 Day TBill TR "], axis=1).mean().values)

1.2861619990880717

In [ ]:
import plotly.express as px

In [ ]:
df1 = df.drop(["Date","Period","U.S. 30 Day TBill TR "],axis=1)

In [ ]:
fig = px.imshow(df1.corr(), # You can use df.corr() to compute the correlation matrix
                x=df1.columns,
                y=df1.columns,
                zmin=-1, # Set the color scale range from -1 to 1 for correlation values
                zmax=1,
                color_continuous_scale='RdBu_r',
                text_auto=True,# Choose a color scale    e the correlation matrix as text annotations
                labels=dict(color='Correlation'))


fig.update_layout(
    title="Heatmap with Text Annotations",
    xaxis_title="X",
    yaxis_title="Y"
)

fig.show()

In [ ]:
(
    pd.melt(
        df.query("Period=='1980s'").drop("Period", axis=1),
        id_vars=["Date"],
        var_name="Indexes",
        value_name="Value",
    )
    .sort_values(by=["Date", "Indexes"])
    .groupby("Indexes")
    .agg(
        Monthly_average_return=("Value", "mean"),
        Monthly_standard_deviation=("Value", "std"),
    )
    .reset_index()
    .assign(Annualized_return=lambda x: x["Monthly_average_return"] * 12)
    .assign(Annualized_standard_deviation=lambda x: x["Monthly_standard_deviation"] * 12 ** 0.5)
)


In [5]:
test_df = df.query("Period=='2000s'").drop(["Date","Period","U.S. 30 Day TBill TR "],axis=1)
test_df

,Russell 2000 TR,S&P 500 TR,LB LT Gvt/Credit TR,MSCI EAFE TR
240,-0.011054,-0.015574,-0.022782,-0.057837
241,-0.028441,-0.020766,0.010098,0.012011
242,0.079044,0.036739,0.006827,0.055824
243,0.008313,-0.061418,-0.050435,0.015237
244,-0.045431,-0.009081,0.036252,0.005331
...,...,...,...,...
355,-0.088102,-0.056791,0.045989,-0.087549
356,-0.113716,-0.071762,0.100108,-0.108083
357,0.150428,0.107723,0.138029,0.096275
358,-0.004912,-0.005059,-0.037082,-0.021764


In [ ]:
mus = test_df.mean()
mus

In [ ]:
cov = test_df.cov()
cov

In [ ]:
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.expected_returns import ema_historical_return
from pypfopt.risk_models import exp_cov
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.plotting import plot_efficient_frontier
from pypfopt.plotting import plot_weights
from pypfopt.cla import CLA

In [ ]:
mu = expected_returns.ema_historical_return(test_df, returns_data = True,compounding=False, span=12)
Sigma = risk_models.exp_cov(test_df, returns_data = True, span=12)

In [ ]:
mu

In [ ]:
Sigma

In [ ]:
ef = EfficientFrontier(mu, Sigma)
ef.min_volatility()

In [ ]:
ef = EfficientFrontier(mu, Sigma)
ef.max_sharpe()

In [ ]:
ret_ef = np.arange(0, 1, 0.001)
vol_ef = []
for i in np.arange(0, 1, 0.001):
    ef = EfficientFrontier(mu, Sigma)
    ef.efficient_return(i)
    vol_ef.append(ef.portfolio_performance()[1])

ef = EfficientFrontier(mu, Sigma)
ef.min_volatility()
min_vol_ret = ef.portfolio_performance()[0]
min_vol_vol = ef.portfolio_performance()[1]

ef = EfficientFrontier(mu, Sigma)
ef.max_sharpe()
max_sharpe_ret = ef.portfolio_performance()[0]
max_sharpe_vol = ef.portfolio_performance()[1]

In [ ]:
min_vol_ret

In [ ]:
min_vol_vol

In [ ]:
ef = EfficientFrontier(mu, Sigma)
ef.min_volatility()
ef.portfolio_performance(verbose=True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

fig, ax = plt.subplots(figsize = [15,10])

sns.lineplot(x = vol_ef, y = ret_ef, label = "Efficient Frontier", ax = ax, color = "b")
# sns.scatterplot(x = [min_vol_vol], y = [min_vol_ret], ax = ax, label = "Minimum Variance Portfolio", color = "purple", s = 100)
# sns.scatterplot(x = [max_sharpe_vol], y = [max_sharpe_ret], ax = ax, label = "Maximum Sharpe Portfolio", color = "green", s = 100)
# sns.lineplot(x = [0, max_sharpe_vol, 1], y = [0.009, max_sharpe_ret, 3.096], label = "Capital Market Line", ax = ax, color = "r")

# ax.set(xlim = [0, 0.4])
# ax.set(ylim = [0, 1])
ax.set_xlabel("Volatility")
ax.set_ylabel("Mean Return")
plt.legend(fontsize='large')
plt.title("Efficient Frontier", fontsize = '20')

In [ ]:
mus = test_df.mean()*12
Sigma = test_df.cov()*12

In [ ]:
#- How many assests to include in each portfolio
n_assets = 4
#-- How many portfolios to generate
n_portfolios = 1000

#-- Initialize empty list to store mean-variance pairs for plotting
mean_variance_pairs = []

np.random.seed(75)
#-- Loop through and generate lots of random portfolios
for i in range(n_portfolios):
    #- Choose assets randomly without replacement
    assets = np.random.choice(list(test_df.columns), n_assets, replace=False)
    #- Choose weights randomly
    weights = np.random.rand(n_assets)
    #- Ensure weights sum to 1
    weights = weights/sum(weights)

    #-- Loop over asset pairs and compute portfolio return and variance
    #- https://quant.stackexchange.com/questions/43442/portfolio-variance-explanation-for-equation-investments-by-zvi-bodie
    portfolio_E_Variance = 0
    portfolio_E_Return = 0
    for i in range(len(assets)):
        portfolio_E_Return += weights[i] * mus.loc[assets[i]]
        for j in range(len(assets)):
            #-- Add variance/covariance for each asset pair
            #- Note that when i==j this adds the variance
            portfolio_E_Variance += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]
            
    #-- Add the mean/variance pairs to a list for plotting
    mean_variance_pairs.append([portfolio_E_Return, portfolio_E_Variance])

In [ ]:
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff
from tqdm import tqdm

In [ ]:
#-- Plot the risk vs. return of randomly generated portfolios
#-- Convert the list from before into an array for easy plotting
mean_variance_pairs = np.array(mean_variance_pairs)

risk_free_rate=0 #-- Include risk free rate here

fig = go.Figure()
fig.add_trace(go.Scatter(x=mean_variance_pairs[:,1]**0.5, y=mean_variance_pairs[:,0], 
                      marker=dict(color=(mean_variance_pairs[:,0]-risk_free_rate)/(mean_variance_pairs[:,1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers'))
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  width=850,
                  height=500)
# fig.update_xaxes(range=[0.18, 0.32])
# fig.update_yaxes(range=[0.02,0.27])
fig.update_layout(coloraxis_colorbar=dict(title="Sharpe Ratio"))

In [ ]:
#-- Create random portfolio weights and indexes
#- How many assests in the portfolio
n_assets = 4

mean_variance_pairs = []
weights_list=[]
tickers_list=[]

for i in tqdm(range(10000)):
    next_i = False
    while True:
        #- Choose assets randomly without replacement
        assets = np.random.choice(list(test_df.columns), n_assets, replace=False)
        #- Choose weights randomly ensuring they sum to one
        weights = np.random.rand(n_assets)
        weights = weights/sum(weights)

        #-- Loop over asset pairs and compute portfolio return and variance
        portfolio_E_Variance = 0
        portfolio_E_Return = 0
        for i in range(len(assets)):
            portfolio_E_Return += weights[i] * mus.loc[assets[i]]
            for j in range(len(assets)):
                portfolio_E_Variance += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]

        #-- Skip over dominated portfolios
        for R,V in mean_variance_pairs:
            if (R > portfolio_E_Return) & (V < portfolio_E_Variance):
                next_i = True
                break
        if next_i:
            break

        #-- Add the mean/variance pairs to a list for plotting
        mean_variance_pairs.append([portfolio_E_Return, portfolio_E_Variance])
        weights_list.append(weights)
        tickers_list.append(assets)
        break

In [ ]:
mean_variance_pairs = np.array(mean_variance_pairs)

risk_free_rate=0 #-- Include risk free rate here

fig = go.Figure()
fig.add_trace(go.Scatter(x=mean_variance_pairs[:,1]**0.5, y=mean_variance_pairs[:,0], 
                      marker=dict(color=(mean_variance_pairs[:,0]-risk_free_rate)/(mean_variance_pairs[:,1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers',
                      text=[str(np.array(tickers_list[i])) + "<br>" + str(np.array(weights_list[i]).round(2)) for i in range(len(tickers_list))]))
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  width=850,
                  height=500)
# fig.update_xaxes(range=[0.18, 0.35])
# fig.update_yaxes(range=[0.05,0.29])
fig.update_layout(coloraxis_colorbar=dict(title="Sharpe Ratio"))

In [ ]:
import numpy as np
from scipy.optimize import minimize

# Define your expected returns and covariance matrix
# You should replace these with your own data
expected_returns = df.query("Period=='2000s'").drop(["Date","Period","U.S. 30 Day TBill TR "],axis=1).mean()
cov_matrix = df.query("Period=='2000s'").drop(["Date","Period", "U.S. 30 Day TBill TR "],axis=1).cov()

# Define the target portfolio mean return
target_mean_return = 5.5

# Define the objective function to minimize standard deviation
def portfolio_volatility(weights):
    portfolio_return = np.sum(expected_returns * weights)
    portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return portfolio_stddev

# Define the constraint for the portfolio mean return
def return_constraint(weights):
    return np.sum(expected_returns * weights) - target_mean_return

# Initial guess for portfolio weights (uniform allocation)
initial_weights = np.array([1.0 / len(expected_returns)] * len(expected_returns))

# Define the optimization problem
constraints = ({'type': 'eq', 'fun': return_constraint})
bounds = tuple((0, 1) for _ in range(len(expected_returns)))  # Bounds for weights (0 <= weight <= 1)
optimal_weights = minimize(portfolio_volatility, initial_weights, method='SLSQP', constraints=constraints, bounds=bounds)

# Extract the optimal portfolio weights
optimal_weights = optimal_weights.x

# Calculate the minimum unconstrained standard deviation
min_std_dev = portfolio_volatility(optimal_weights)

# Calculate the portfolio mean return for the minimum unconstrained standard deviation
portfolio_mean_return = np.sum(expected_returns * optimal_weights)

print("Minimum Unconstrained Standard Deviation:", min_std_dev)
print("Portfolio Mean Return:", portfolio_mean_return)


In [ ]:
expected_returns.index.tolist()

In [ ]:
expected_returns = df.query("Period=='2000s'").drop(["Date","Period","U.S. 30 Day TBill TR "],axis=1).mean()
cov_matrix = df.query("Period=='2000s'").drop(["Date","Period", "U.S. 30 Day TBill TR "],axis=1).cov()
rows = []
target_mean_returns = [5.5, 8.2, 11.0, 13.9, 15.2, 17.1, 19.7, 21.3, 23.2, 24.8]
# Loop through each target mean return and optimize the portfolio
for target_mean_return in target_mean_returns:
    # Define the constraint function for the current target mean return
    def portfolio_std_deviation(weights,cov_matrix):
        portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        return portfolio_stddev

    # Define the constraint function for target mean return
    def mean_return_constraint(weights):
        return np.sum(expected_returns * weights) - target_mean_return

    # Define the constraint to ensure the weights add up to 1 (fully invested)
    def budget_constraint(weights):
        return np.sum(weights) - 1

    # Define the initial guess for portfolio weights
    initial_weights = np.ones(len(expected_returns)) / len(expected_returns)

    # Define bounds for the weights (0 to 1, meaning fully invested or not invested)
    bounds = [(0, 1) for _ in expected_returns]

    # Define the optimization problem with both constraints
    constraints = [{'type': 'eq', 'fun': mean_return_constraint},
                   {'type': 'eq', 'fun': budget_constraint}]

    # Use the minimize function to solve the optimization problem
    optimal_weights = minimize(
        portfolio_std_deviation,         # Objective function to minimize
        initial_weights, 
        cov_matrix,   
        method='SLSQP',                 # Sequential Least Squares Quadratic Programming
        constraints=constraints,         # Constraints
        bounds=bounds                    # Bounds for asset weights
    )

    # Extract the optimized portfolio weights
    optimized_weights = optimal_weights.x

    # Calculate the minimum standard deviation for the current target
    min_std_dev = portfolio_std_deviation(optimized_weights, cov_matrix)

    # Print the results for the current target
    print(f"Target Mean Return: {target_mean_return}")
    print("Optimal Portfolio Weights:", optimized_weights)
    print("Minimum Portfolio Standard Deviation:", min_std_dev)
    print("-" * 40)  # Separator for clarity
    row = [min_std_dev] + optimized_weights.tolist() + [np.dot(expected_returns, optimized_weights)]
    rows.append(row)
    


In [ ]:
df = pd.DataFrame(rows,columns=["min_std_dev"]+expected_returns.index.tolist()+["portfolio_mean_return"])

In [ ]:
pd.concat(
    [pd.DataFrame(target_mean_returns,columns=["target_mean_returns"]),
    df],
    axis=1
)

In [ ]:
np.dot(expected_returns, optimized_weights)

In [ ]:
optimal_weights.x

In [ ]:
import numpy as np
import random

# Define your expected returns and covariance matrix
# You should replace these with your own data
expected_returns = df.query("Period=='2000s'").drop(["Date","Period"],axis=1).mean()
cov_matrix = df.query("Period=='2000s'").drop(["Date","Period"],axis=1).cov()

# Define the target portfolio mean return
target_mean_return = 5.5

# Genetic Algorithm Parameters
population_size = 10000
num_generations = 100
crossover_prob = 0.7
mutation_prob = 0.2

# Define the chromosome length (number of assets)
chromosome_length = len(expected_returns)

# Define the fitness function
def portfolio_fitness(weights):
    portfolio_return = np.sum(expected_returns * weights)
    portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return portfolio_stddev,  # Note the comma to return a tuple

# Initialize the population with random portfolios
population = [np.random.rand(chromosome_length) for _ in range(population_size)]

for generation in range(num_generations):
    print("Generation %d..." % generation)
    # Evaluate the fitness of each portfolio in the population
    fitness_scores = [portfolio_fitness(weights) for weights in population]

    # Select portfolios for crossover
    selected_indices = np.random.choice(population_size, size=population_size, replace=True)
    selected_portfolios = [population[i] for i in selected_indices]

    # Perform crossover
    offspring = []
    for i in range(0, population_size, 2):
        parent1, parent2 = selected_portfolios[i], selected_portfolios[i + 1]
        if random.random() < crossover_prob:
            crossover_point = random.randint(1, chromosome_length - 1)
            child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
        else:
            child1, child2 = parent1.copy(), parent2.copy()
        offspring.extend([child1, child2])

    # Perform mutation
    for i in range(population_size):
        if random.random() < mutation_prob:
            mutation_point = random.randint(0, chromosome_length - 1)
            population[i][mutation_point] = random.random()

    # Replace the old population with the new population
    population = offspring

# Find the best portfolio in the final population
best_portfolio = min(population, key=lambda weights: portfolio_fitness(weights))

# Calculate the portfolio mean return and standard deviation
best_portfolio_return = np.sum(expected_returns * best_portfolio)
best_portfolio_stddev = np.sqrt(np.dot(best_portfolio.T, np.dot(cov_matrix, best_portfolio)))

print("Best Portfolio Weights:", best_portfolio)
print("Best Portfolio Mean Return:", best_portfolio_return)
print("Best Portfolio Standard Deviation:", best_portfolio_stddev)


In [18]:
expected_returns = test_df.mean()*12
expected_returns.values.tolist()

[0.06416920407599999, 0.021953809847000016, 0.0902087, 0.06450622529000001]

In [19]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

# Define the problem as a minimization problem
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

# Define the problem-specific functions and constraints

# Dummy data for expected returns and covariance matrix
expected_returns = [0.06416920407599999, 0.021953809847000016, 0.0902087, 0.06450622529000001]
cov_matrix = [[4.43902182e-02, 3.03961209e-02, 1.63717276e-03, 3.41134446e-02],
       [3.03961209e-02, 2.53635406e-02, 1.00647360e-03, 2.76667501e-02],
       [1.63717276e-03, 1.00647360e-03, 2.24798104e-02, 5.95227542e-05],
       [3.41134446e-02, 2.76667501e-02, 5.95227542e-05, 3.76493663e-02]
]

target_mean_return = 5.5

def evaluate_portfolio(individual):
    # Calculate portfolio return and standard deviation
    portfolio_return = sum(expected_returns[i] * w for i, w in enumerate(individual))
    portfolio_cov_matrix = np.array([[cov_matrix[i][j] * w1 * w2 for i, w1 in enumerate(individual)] for j, w2 in enumerate(individual)])
    portfolio_stddev = np.sqrt(np.dot(individual, np.dot(portfolio_cov_matrix, individual)))

    # Penalize portfolios that do not meet constraints
    deviation_from_target = abs(portfolio_return - target_mean_return)
    penalty = deviation_from_target  # Adjust penalty as needed

    return portfolio_stddev + penalty,

# Genetic Algorithm Parameters
population_size = 100000
num_generations = 1000
crossover_prob = 0.5
mutation_prob = 0.1
chromosome_length = 4  # Number of assets

# Create a toolbox for the GA
toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, 0, 1)  # Gene values (weights)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, chromosome_length)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate_portfolio)
toolbox.register("mate", tools.cxBlend, alpha=0.5)  # Blend crossover
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.2, indpb=mutation_prob)  # Gaussian mutation
toolbox.register("select", tools.selTournament, tournsize=3)

# Create an initial population
population = toolbox.population(n=population_size)

# Run the genetic algorithm
final_population, _ = algorithms.eaMuPlusLambda(population, toolbox, mu=population_size, lambda_=population_size, 
                                                cxpb=crossover_prob, mutpb=mutation_prob, ngen=num_generations,
                                                stats=None, halloffame=None, verbose=True)

# Select the best portfolio from the final population
best_portfolio = tools.selBest(final_population, k=1)[0]

# Print the results
print("Best Portfolio Weights:", best_portfolio)
print("Best Portfolio Mean Return:", sum(expected_returns[i] * best_portfolio[i] for i in range(len(best_portfolio))))
print("Best Portfolio Standard Deviation:", evaluate_portfolio(best_portfolio)[0])


/Users/hanyuwu/Library/Caches/pypoetry/virtualenvs/asset-allocation-project-ax0yMTeZ-py3.11/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/hanyuwu/Library/Caches/pypoetry/virtualenvs/asset-allocation-project-ax0yMTeZ-py3.11/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	100000
1  	60205 
2  	59926 
3  	60024 
4  	60124 
5  	59921 
6  	60170 
7  	60177 
8  	60221 
9  	59840 
10 	60027 
11 	59966 
12 	59897 
13 	59998 
14 	59978 
15 	60009 
16 	60262 
17 	60251 
18 	60033 
19 	59770 
20 	59968 
21 	60193 
22 	60128 
23 	60274 
24 	59786 
25 	60022 
26 	60169 
27 	60248 
28 	60178 
29 	59922 
30 	60056 
31 	60008 
32 	59875 
33 	60102 
34 	60024 
35 	59957 
36 	59903 
37 	59905 
38 	59918 
39 	59810 
40 	59933 
41 	60182 
42 	59918 
43 	60059 
44 	60149 
45 	60031 
46 	60022 
47 	59905 
48 	60120 
49 	59753 
50 	60215 
51 	60102 
52 	59728 
53 	60235 
54 	59810 
55 	59977 
56 	60169 
57 	59763 
58 	60291 
59 	60007 
60 	60147 
61 	60053 
62 	60115 
63 	59834 
64 	60149 
65 	59912 
66 	59677 
67 	60021 
68 	60102 
69 	59855 
70 	59756 
71 	59969 
72 	59865 
73 	59865 
74 	60057 
75 	60176 
76 	59761 
77 	59884 
78 	60373 
79 	59883 


KeyboardInterrupt: 

In [20]:
best_portfolio = tools.selBest(final_population, k=1)[0]

# Print the results
print("Best Portfolio Weights:", best_portfolio)
print("Best Portfolio Mean Return:", sum(expected_returns[i] * best_portfolio[i] for i in range(len(best_portfolio))))
print("Best Portfolio Standard Deviation:", evaluate_portfolio(best_portfolio)[0])

Best Portfolio Weights: [0.22356607475700002, 0.10254134508346303, 0.4088421200492207, 0.2540224261302224]
Best Portfolio Mean Return: 0.0698643742705228
Best Portfolio Standard Deviation: 5.465067813900535


In [1]:
pd.read_csv("../data/processed/processed_data.csv")

NameError: name 'pd' is not defined